In [1]:
import pandas as pd
import warnings
import time
import random
from tqdm import tqdm
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
def req(url):
    import requests
    from bs4 import BeautifulSoup as bs4
    response = requests.get(url)
    contents = response.content
    soup = bs4(contents, "html.parser")
    return soup

In [3]:
def fill_area(row):
    if row["adresse"] == "":
        return row["area"]
    else:
        return row["adresse"]

In [4]:
def rent_bin(row):
    if row["rent"] < 600:
        return "below 600"
    elif (row["rent"] >= 600) & (row["rent"] < 1000):
        return "600 to 1000"
    elif (row["rent"] >= 1000) & (row["rent"] < 1500):
        return "1000 to 1500"
    elif (row["rent"] >= 1500) & (row["rent"] < 2000):
        return "1500 to 2000"
    elif (row["rent"] >= 2000) & (row["rent"] < 3000):
        return "2000 to 3000"
    else:
        return "over 3000"

In [5]:
df = pd.read_csv("data raw.csv")
df["adresse"] = df["adresse"].str.replace("berlin","Berlin")
adresse,area = [],[]
for i in df["adresse"]:
    adresse.append(i.split("Berlin ")[0].strip(" "))
    try:
        area.append(i.split("(")[1].strip(" "))
    except:
        area.append("")
size = []
for i in df["size"]:
    size.append(i.split(" m")[0])
room = []
for i in df["rooms"]:
    room.append(i.split(" Z")[0])
df["rooms"] = room
df["rooms"] = df["rooms"].astype(float)
df["size"] = size
df["size"] = df["size"].astype(float)
df["adresse"] = adresse
df["area"] = area
df["adresse"] = df["adresse"].str.replace(",","")
df["rent"] = df["rent"].str.replace("€","")
df["rent"] = df["rent"].str.replace(".","")
df["rent"] = df["rent"].str.replace(",",".")
df["rent"] = df["rent"].astype(float)
df["area"] = df["area"].str.replace("(","").str.replace(")","").str.replace("/ ","")
df["adresse"] = df.apply(fill_area, axis = 1)
df["rent_range"] = df.apply(rent_bin, axis = 1)
df.to_csv("data clean.csv", index = False)

In [334]:
df["link"][df["rent"]==df["rent"].max()].iloc[0]

'https://www.immowelt.de/expose/285rw59'

In [335]:
df["link"].iloc[0]

'https://www.immowelt.de/expose/29btl59'

In [336]:
df.shape[0]

500

## DONT RUN THIS CELL

In [328]:
df0 = pd.DataFrame()
for j in tqdm(range(df.shape[0])):
    time.sleep(random.randint(4,7))
    soup = req(df["link"].iloc[j])
    infobox = soup.find("app-estate-object-informations")
    d = {}
    try:
        infos = infobox.find_all("li",{"class":"ng-star-inserted"})
        for i in infos:
            i = i.text
            if ":" in i:
                i1 = i.split(":")[0]
                i2 = i.split(":")[1].strip()
                d[i1] = [i2]
            else:
                d[i] = [1]
    except:
        d["not available"] = [1]
    df1 = pd.DataFrame.from_dict(d)
    df0 = pd.concat([df0,df1])
df0.shape

  1%|          | 5/500 [00:35<57:54,  7.02s/it]


KeyboardInterrupt: 

In [337]:
display(df0.head(10))

,not available,Bad mit Fenster,Balkon,Einbauküche,Ausstattung,Zustand,Haustiere erlaubt,Personenaufzug
0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,1.0,1.0,1.0,möbliert,renoviert / saniert,1.0,1.0
0,NaN,NaN,NaN,1.0,möbliert,renoviert / saniert,NaN,NaN
0,NaN,NaN,NaN,1.0,möbliert,renoviert / saniert,1.0,1.0


In [6]:
# df0 = df0.reset_index(drop = True)
df_raw = pd.concat([df,df0], axis = 1)
try:
    df_raw = df_raw.drop(columns = ["level_0"])
except:
    pass
df_raw.head()

NameError: name 'df0' is not defined

In [ ]:
df_raw.to_csv("data full raw.csv", index = False)

## START FROM HERE

In [7]:
df_raw = pd.read_csv("data full raw.csv")

### def cleaning (1)

In [8]:
def guest_wc(df):
    import numpy as np
    el1,el2,el3,el4,el5,el6,el7,el8,el9 = [],[],[],[],[],[],[],[],[]
    for i in df["Bad mit Wanne, Gäste-WC "]:
        if i == 1.0:
            el1.append(1)
        else:
            el1.append(0)
    for i in df["Bad mit Wanne und Dusche, Gäste-WC "]:
        if i == 1.0:
            el2.append(1)
        else:
            el2.append(0)
    for i in df['Bad mit Fenster und Wanne, Gäste-WC ']:
        if i == 1.0:
            el3.append(1)
        else:
            el3.append(0)
    for i in df['Bad mit Dusche, Gäste-WC ']:
        if i == 1.0:
            el4.append(1)
        else:
            el4.append(0)
    for i in df['Bad mit Fenster, Wanne und Dusche, Gäste-WC ']:
        if i == 1.0:
            el5.append(1)
        else:
            el5.append(0)
    for i in df['Gäste-WC ']:
        if i == 1.0:
            el6.append(1)
        else:
            el6.append(0)
    for i in df['Bad mit Fenster, Gäste-WC ']:
        if i == 1.0:
            el7.append(1)
        else:
            el7.append(0)
    for i in df['Bad mit Fenster, Wanne und Dusche, Gäste-WC, Bad/WC getrennt ']:
        if i == 1.0:
            el8.append(1)
        else:
            el8.append(0)
    for i in df['Bad mit Fenster und Dusche, Gäste-WC ']:
        if i == 1.0:
            el9.append(1)
        else:
            el9.append(0)
    el1 = np.array(el1)
    el2 = np.array(el2)
    el3 = np.array(el3)
    el4 = np.array(el4)
    el5 = np.array(el5)
    el6 = np.array(el6)
    el7 = np.array(el7)
    el8 = np.array(el8)
    el9 = np.array(el9)
    el0 = el1+el2+el3+el4+el5+el6+el7+el8+el9
    el = []
    for i in el0:
        if i > 0:
            el.append(1)
        else:
            el.append(0)
    return el

In [9]:
def shower(df):
    import numpy as np
    el1,el2,el3,el4,el5,el6,el7,el8,el9 = [],[],[],[],[],[],[],[],[]
    for i in df["Bad mit Wanne und Dusche "]:
        if i == 1.0:
            el1.append(1)
        else:
            el1.append(0)
    for i in df['Bad mit Dusche ']:
        if i == 1.0:
            el2.append(1)
        else:
            el2.append(0)
    for i in df['Bad mit Wanne und Dusche, Gäste-WC ']:
        if i == 1.0:
            el3.append(1)
        else:
            el3.append(0)
    for i in df['Bad mit Dusche, Gäste-WC ']:
        if i == 1.0:
            el4.append(1)
        else:
            el4.append(0)
    for i in df['Bad mit Fenster, Wanne und Dusche, Gäste-WC ']:
        if i == 1.0:
            el5.append(1)
        else:
            el5.append(0)
    for i in df['Bad mit Fenster, Wanne und Dusche ']:
        if i == 1.0:
            el6.append(1)
        else:
            el6.append(0)
    for i in df['Bad mit Fenster, Gäste-WC ']:
        if i == 1.0:
            el7.append(1)
        else:
            el7.append(0)
    for i in df['Bad mit Fenster, Wanne und Dusche, Gäste-WC, Bad/WC getrennt ']:
        if i == 1.0:
            el8.append(1)
        else:
            el8.append(0)
    for i in df['Bad mit Fenster und Dusche, Gäste-WC ']:
        if i == 1.0:
            el9.append(1)
        else:
            el9.append(0)
    el1 = np.array(el1)
    el2 = np.array(el2)
    el3 = np.array(el3)
    el4 = np.array(el4)
    el5 = np.array(el5)
    el6 = np.array(el6)
    el7 = np.array(el7)
    el8 = np.array(el8)
    el9 = np.array(el9)
    el0 = el1+el2+el3+el4+el5+el6+el7+el8+el9
    el = []
    for i in el0:
        if i > 0:
            el.append(1)
        else:
            el.append(0)
    return el

In [10]:
def bathtub(df):
    import numpy as np
    el1,el2,el3,el4,el5,el6,el7,el8,el9 = [],[],[],[],[],[],[],[],[]
    for i in df["Bad mit Wanne und Dusche "]:
        if i == 1.0:
            el1.append(1)
        else:
            el1.append(0)
    for i in df['Bad mit Wanne, Gäste-WC ']:
        if i == 1.0:
            el2.append(1)
        else:
            el2.append(0)
    for i in df['Bad mit Fenster und Wanne ']:
        if i == 1.0:
            el3.append(1)
        else:
            el3.append(0)
    for i in df['Bad mit Wanne und Dusche, Gäste-WC ']:
        if i == 1.0:
            el4.append(1)
        else:
            el4.append(0)
    for i in df['Bad mit Fenster und Wanne, Gäste-WC ']:
        if i == 1.0:
            el5.append(1)
        else:
            el5.append(0)
    for i in df['Bad mit Wanne ']:
        if i == 1.0:
            el6.append(1)
        else:
            el6.append(0)
    for i in df['Bad mit Fenster, Wanne und Dusche, Gäste-WC ']:
        if i == 1.0:
            el7.append(1)
        else:
            el7.append(0)
    for i in df['Bad mit Fenster, Wanne und Dusche ']:
        if i == 1.0:
            el8.append(1)
        else:
            el8.append(0)
    for i in df['Bad mit Fenster, Wanne und Dusche, Gäste-WC, Bad/WC getrennt ']:
        if i == 1.0:
            el9.append(1)
        else:
            el9.append(0)
    el1 = np.array(el1)
    el2 = np.array(el2)
    el3 = np.array(el3)
    el4 = np.array(el4)
    el5 = np.array(el5)
    el6 = np.array(el6)
    el7 = np.array(el7)
    el8 = np.array(el8)
    el9 = np.array(el9)
    el0 = el1+el2+el3+el4+el5+el6+el7+el8+el9
    el = []
    for i in el0:
        if i > 0:
            el.append(1)
        else:
            el.append(0)
    return el

In [11]:
def bathroom_window(df):
    import numpy as np
    el1,el2,el3,el4,el5,el6,el7,el8,el9 = [],[],[],[],[],[],[],[],[]
    for i in df['Bad mit Fenster ']:
        if i == 1.0:
            el1.append(1)
        else:
            el1.append(0)
    for i in df['Bad mit Fenster und Wanne ']:
        if i == 1.0:
            el2.append(1)
        else:
            el2.append(0)
    for i in df['Bad mit Fenster und Wanne, Gäste-WC ']:
        if i == 1.0:
            el3.append(1)
        else:
            el3.append(0)
    for i in df['Bad mit Fenster, Wanne und Dusche, Gäste-WC ']:
        if i == 1.0:
            el4.append(1)
        else:
            el4.append(0)
    for i in df['Bad mit Fenster und Dusche ']:
        if i == 1.0:
            el5.append(1)
        else:
            el5.append(0)
    for i in df['Bad mit Fenster, Wanne und Dusche ']:
        if i == 1.0:
            el6.append(1)
        else:
            el6.append(0)
    for i in df['Bad mit Fenster, Gäste-WC ']:
        if i == 1.0:
            el7.append(1)
        else:
            el7.append(0)
    for i in df['Bad mit Fenster, Wanne und Dusche, Gäste-WC, Bad/WC getrennt ']:
        if i == 1.0:
            el8.append(1)
        else:
            el8.append(0)
    for i in df['Bad mit Fenster und Dusche, Gäste-WC ']:
        if i == 1.0:
            el9.append(1)
        else:
            el9.append(0)
    el1 = np.array(el1)
    el2 = np.array(el2)
    el3 = np.array(el3)
    el4 = np.array(el4)
    el5 = np.array(el5)
    el6 = np.array(el6)
    el7 = np.array(el7)
    el8 = np.array(el8)
    el9 = np.array(el9)
    el0 = el1+el2+el3+el4+el5+el6+el7+el8+el9
    el = []
    for i in el0:
        if i > 0:
            el.append(1)
        else:
            el.append(0)
    return el

### cleaning (1)

In [12]:
lguest_wc = []
lguest_wc = guest_wc(df_raw)
lshower = []
lshower = shower(df_raw)
lbathtub = []
lbathtub = bathtub(df_raw)
lbathroom_window = []
lbathroom_window = bathroom_window(df_raw)
df_new = df_raw.copy()
df_new["guest_wc"] = lguest_wc
df_new["shower"] = lshower
df_new["bathtub"] = lbathtub
df_new["bathroom_window"] = lbathroom_window


In [13]:
columns_to_drop = ['Bad mit Fenster ','Bad mit Wanne und Dusche ','Bad mit Dusche ','Bad mit Wanne, Gäste-WC ',
                    'Bad mit Fenster und Wanne ','Bad mit Wanne und Dusche, Gäste-WC ','Bad mit Fenster und Wanne, Gäste-WC ',
                    'Bad mit Wanne ','Bad mit Dusche, Gäste-WC ','Bad mit Fenster, Wanne und Dusche, Gäste-WC ',
                    'Gäste-WC ','Bad mit Fenster und Dusche ','Bad mit Fenster, Wanne und Dusche ','Bad mit Fenster, Gäste-WC ',
                    'Bad mit Fenster, Wanne und Dusche, Gäste-WC, Bad/WC getrennt ','Bad mit Fenster und Dusche, Gäste-WC ']
print(df_new.shape[1])
try:
    df_new = df_new.drop(columns = columns_to_drop)
except:
    pass
print(df_new.shape[1])

82
66


### def cleaning (2)

In [16]:
def barrier_free(df):
    import numpy as np
    el1,el2,el3,el4 = [],[],[],[]
    for i in df['barrierefrei, rollstuhlgerecht ']:
        if i == 1.0:
            el1.append(1)
        else:
            el1.append(0)
    for i in df['barrierefrei ']:
        if i == 1.0:
            el2.append(1)
        else:
            el2.append(0)
    for i in df['barrierefrei, rollstuhlgerecht, WG-geeignet ']:
        if i == 1.0:
            el3.append(1)
        else:
            el3.append(0)
    for i in df['barrierefrei, WG-geeignet ']:
        if i == 1.0:
            el4.append(1)
        else:
            el4.append(0)
    el1 = np.array(el1)
    el2 = np.array(el2)
    el3 = np.array(el3)
    el4 = np.array(el4)
    el0 = el1+el2+el3+el4
    el = []
    for i in el0:
        if i > 0:
            el.append(1)
        else:
            el.append(0)
    return el

In [17]:
def wheelchair_friendly(df):
    import numpy as np
    el1,el2,el3 = [],[],[]
    for i in df['barrierefrei, rollstuhlgerecht ']:
        if i == 1.0:
            el1.append(1)
        else:
            el1.append(0)
    for i in df['rollstuhlgerecht ']:
        if i == 1.0:
            el2.append(1)
        else:
            el2.append(0)
    for i in df['barrierefrei, rollstuhlgerecht, WG-geeignet ']:
        if i == 1.0:
            el3.append(1)
        else:
            el3.append(0)

    el1 = np.array(el1)
    el2 = np.array(el2)
    el3 = np.array(el3)

    el0 = el1+el2+el3
    el = []
    for i in el0:
        if i > 0:
            el.append(1)
        else:
            el.append(0)
    return el

In [18]:
def apartment_sharing(df):
    import numpy as np
    el1,el2,el3 = [],[],[]
    for i in df['barrierefrei, rollstuhlgerecht, WG-geeignet ']:
        if i == 1.0:
            el1.append(1)
        else:
            el1.append(0)
    for i in df['barrierefrei, WG-geeignet ']:
        if i == 1.0:
            el2.append(1)
        else:
            el2.append(0)
    for i in df['WG-geeignet ']:
        if i == 1.0:
            el3.append(1)
        else:
            el3.append(0)

    el1 = np.array(el1)
    el2 = np.array(el2)
    el3 = np.array(el3)

    el0 = el1+el2+el3
    el = []
    for i in el0:
        if i > 0:
            el.append(1)
        else:
            el.append(0)
    return el

In [19]:
def balcony(df):
    import numpy as np
    el1,el2,el3,el4,el5,el6,el7,el8,el9 = [],[],[],[],[],[],[],[],[]
    for i in df['Balkon ']:
        if i == 1.0:
            el1.append(1)
        else:
            el1.append(0)
    for i in df['Balkon, Garten, Gartenmitbenutzung ']:
        if i == 1.0:
            el2.append(1)
        else:
            el2.append(0)
    for i in df['Balkon, Terrasse ']:
        if i == 1.0:
            el3.append(1)
        else:
            el3.append(0)
    for i in df['Balkon, Garten ']:
        if i == 1.0:
            el4.append(1)
        else:
            el4.append(0)
    for i in df['Balkon, Terrasse, Gartenmitbenutzung ']:
        if i == 1.0:
            el5.append(1)
        else:
            el5.append(0)
    for i in df['Balkon, Gartenmitbenutzung ']:
        if i == 1.0:
            el6.append(1)
        else:
            el6.append(0)
    for i in df['Balkon, Garten, Terrasse ']:
        if i == 1.0:
            el7.append(1)
        else:
            el7.append(0)
    for i in df['Balkon, Garten, Terrasse, Gartenanteil ']:
        if i == 1.0:
            el8.append(1)
        else:
            el8.append(0)
    for i in df['Balkon, Wintergarten ']:
        if i == 1.0:
            el9.append(1)
        else:
            el9.append(0)
    el1 = np.array(el1)
    el2 = np.array(el2)
    el3 = np.array(el3)
    el4 = np.array(el4)
    el5 = np.array(el5)
    el6 = np.array(el6)
    el7 = np.array(el7)
    el8 = np.array(el8)
    el9 = np.array(el9)
    el0 = el1+el2+el3+el4+el5+el6+el7+el8+el9
    el = []
    for i in el0:
        if i > 0:
            el.append(1)
        else:
            el.append(0)
    return el

In [20]:
def garden(df):
    import numpy as np
    el1,el2,el3,el4,el5,el6,el7,el8,el9,el10,el11,el12,el13 = [],[],[],[],[],[],[],[],[],[],[],[],[]
    for i in df['Garten ']:
        if i == 1.0:
            el1.append(1)
        else:
            el1.append(0)
    for i in df['Balkon, Garten, Gartenmitbenutzung ']:
        if i == 1.0:
            el2.append(1)
        else:
            el2.append(0)
    for i in df['Gartenmitbenutzung ']:
        if i == 1.0:
            el3.append(1)
        else:
            el3.append(0)
    for i in df['Terrasse, Gartenmitbenutzung ']:
        if i == 1.0:
            el4.append(1)
        else:
            el4.append(0)
    for i in df['Balkon, Garten ']:
        if i == 1.0:
            el5.append(1)
        else:
            el5.append(0)
    for i in df['Garten, Terrasse ']:
        if i == 1.0:
            el6.append(1)
        else:
            el6.append(0)
    for i in df['Balkon, Terrasse, Gartenmitbenutzung ']:
        if i == 1.0:
            el7.append(1)
        else:
            el7.append(0)
    for i in df['Balkon, Gartenmitbenutzung ']:
        if i == 1.0:
            el8.append(1)
        else:
            el8.append(0)
    for i in df['Garten, Terrasse, Gartenmitbenutzung ']:
        if i == 1.0:
            el9.append(1)
        else:
            el9.append(0)
    for i in df['Balkon, Garten, Terrasse ']:
        if i == 1.0:
            el10.append(1)
        else:
            el10.append(0)
    for i in df['Balkon, Garten, Terrasse, Gartenanteil ']:
        if i == 1.0:
            el11.append(1)
        else:
            el11.append(0)
    for i in df['Garten, Terrasse, Gartenanteil ']:
        if i == 1.0:
            el12.append(1)
        else:
            el12.append(0)
    for i in df['Garten, Gartenmitbenutzung ']:
        if i == 1.0:
            el13.append(1)
        else:
            el13.append(0)
    el1 = np.array(el1)
    el2 = np.array(el2)
    el3 = np.array(el3)
    el4 = np.array(el4)
    el5 = np.array(el5)
    el6 = np.array(el6)
    el7 = np.array(el7)
    el8 = np.array(el8)
    el9 = np.array(el9)
    el10 = np.array(el10)
    el11 = np.array(el11)
    el12 = np.array(el12)
    el13 = np.array(el13)
    el0 = el1+el2+el3+el4+el5+el6+el7+el8+el9+el10+el11+el12+el13
    el = []
    for i in el0:
        if i > 0:
            el.append(1)
        else:
            el.append(0)
    return el

In [21]:
def garden_sharing(df):
    import numpy as np
    el1,el2,el3,el4,el5,el6,el7,el8,el9,el10 = [],[],[],[],[],[],[],[],[],[]
    for i in df['Balkon, Garten, Gartenmitbenutzung ']:
        if i == 1.0:
            el1.append(1)
        else:
            el1.append(0)
    for i in df['Gartenmitbenutzung ']:
        if i == 1.0:
            el2.append(1)
        else:
            el2.append(0)
    for i in df['Terrasse, Gartenmitbenutzung ']:
        if i == 1.0:
            el3.append(1)
        else:
            el3.append(0)
    for i in df['Balkon, Terrasse, Gartenmitbenutzung ']:
        if i == 1.0:
            el4.append(1)
        else:
            el4.append(0)
    for i in df['Terrasse, Gartenanteil ']:
        if i == 1.0:
            el5.append(1)
        else:
            el5.append(0)
    for i in df['Balkon, Gartenmitbenutzung ']:
        if i == 1.0:
            el6.append(1)
        else:
            el6.append(0)
    for i in df['Garten, Terrasse, Gartenmitbenutzung ']:
        if i == 1.0:
            el7.append(1)
        else:
            el7.append(0)
    for i in df['Balkon, Garten, Terrasse, Gartenanteil ']:
        if i == 1.0:
            el8.append(1)
        else:
            el8.append(0)
    for i in df['Garten, Terrasse, Gartenanteil ']:
        if i == 1.0:
            el9.append(1)
        else:
            el9.append(0)
    for i in df['Garten, Gartenmitbenutzung ']:
        if i == 1.0:
            el10.append(1)
        else:
            el10.append(0)
    el1 = np.array(el1)
    el2 = np.array(el2)
    el3 = np.array(el3)
    el4 = np.array(el4)
    el5 = np.array(el5)
    el6 = np.array(el6)
    el7 = np.array(el7)
    el8 = np.array(el8)
    el9 = np.array(el9)
    el10 = np.array(el10)
    el0 = el1+el2+el3+el4+el5+el6+el7+el8+el9+el10
    el = []
    for i in el0:
        if i > 0:
            el.append(1)
        else:
            el.append(0)
    return el

In [22]:
def terrace(df):
    import numpy as np
    el1,el2,el3,el4,el5,el6,el7,el8,el9,el10 = [],[],[],[],[],[],[],[],[],[]
    for i in df['Balkon, Terrasse ']:
        if i == 1.0:
            el1.append(1)
        else:
            el1.append(0)
    for i in df['Terrasse, Gartenmitbenutzung ']:
        if i == 1.0:
            el2.append(1)
        else:
            el2.append(0)
    for i in df['Garten, Terrasse ']:
        if i == 1.0:
            el3.append(1)
        else:
            el3.append(0)
    for i in df['Terrasse ']:
        if i == 1.0:
            el4.append(1)
        else:
            el4.append(0)
    for i in df['Balkon, Terrasse, Gartenmitbenutzung ']:
        if i == 1.0:
            el5.append(1)
        else:
            el5.append(0)
    for i in df['Terrasse, Gartenanteil ']:
        if i == 1.0:
            el6.append(1)
        else:
            el6.append(0)
    for i in df['Garten, Terrasse, Gartenmitbenutzung ']:
        if i == 1.0:
            el7.append(1)
        else:
            el7.append(0)
    for i in df['Balkon, Garten, Terrasse ']:
        if i == 1.0:
            el8.append(1)
        else:
            el8.append(0)
    for i in df['Balkon, Garten, Terrasse, Gartenanteil ']:
        if i == 1.0:
            el9.append(1)
        else:
            el9.append(0)
    for i in df['Garten, Terrasse, Gartenanteil ']:
        if i == 1.0:
            el10.append(1)
        else:
            el10.append(0)
    el1 = np.array(el1)
    el2 = np.array(el2)
    el3 = np.array(el3)
    el4 = np.array(el4)
    el5 = np.array(el5)
    el6 = np.array(el6)
    el7 = np.array(el7)
    el8 = np.array(el8)
    el9 = np.array(el9)
    el10 = np.array(el10)
    el0 = el1+el2+el3+el4+el5+el6+el7+el8+el9+el10
    el = []
    for i in el0:
        if i > 0:
            el.append(1)
        else:
            el.append(0)
    return el

### cleaning (2)

In [23]:
lbarrier_free = []
lbarrier_free = barrier_free(df_raw)
lwheelchair_friendly = []
lwheelchair_friendly = wheelchair_friendly(df_raw)
lapartment_sharing = []
lapartment_sharing = apartment_sharing(df_raw)
lbalcony = []
lbalcony = balcony(df_raw)
lgarden = []
lgarden = garden(df_raw)
lgarden_sharing = []
lgarden_sharing = garden_sharing(df_raw)
lterrace = []
lterrace = terrace(df_raw)

In [24]:
df_new["barrier_free"] = lbarrier_free
df_new["wheelchair_friendly"] = lwheelchair_friendly
df_new["apartment_sharing"] = lapartment_sharing

df_new["balcony"] = lbalcony
df_new["garden"] = lgarden
df_new["garden_sharing"] = lgarden_sharing
df_new["terrace"] = lterrace

In [25]:
columns_to_drop = ['barrierefrei, rollstuhlgerecht ','barrierefrei, rollstuhlgerecht, WG-geeignet ',
                    'rollstuhlgerecht ','barrierefrei, WG-geeignet ','barrierefrei ','WG-geeignet ',
                    'Balkon ','Garten ','Balkon, Garten, Gartenmitbenutzung ','Gartenmitbenutzung ',
                    'Balkon, Terrasse ','Terrasse, Gartenmitbenutzung ','Balkon, Garten ','Terrasse ',
                    'Garten, Terrasse ','Balkon, Terrasse, Gartenmitbenutzung ','Terrasse, Gartenanteil ',
                    'Balkon, Gartenmitbenutzung ','Garten, Terrasse, Gartenmitbenutzung ',
                    'Balkon, Garten, Terrasse ','Balkon, Garten, Terrasse, Gartenanteil ',
                    'Garten, Terrasse, Gartenanteil ','Garten, Gartenmitbenutzung ','Balkon, Wintergarten ']
print(df_new.shape[1])
try:
    df_new = df_new.drop(columns = columns_to_drop)
except:
    pass
print(df_new.shape[1])

73
49


### def cleaning (3)

In [26]:
def pantry(df):
    import numpy as np
    el1,el2,el3,el4 = [],[],[],[]
    for i in df['Pantry ']:
        if i == 1.0:
            el1.append(1)
        else:
            el1.append(0)
    for i in df['Pantry, Speisekammer ']:
        if i == 1.0:
            el2.append(1)
        else:
            el2.append(0)
    for i in df['Einbauküche, offene Küche, Pantry, Speisekammer ']:
        if i == 1.0:
            el3.append(1)
        else:
            el3.append(0)
    for i in df['Einbauküche, offene Küche, Pantry, Speisekammer ']:
        if i == 1.0:
            el4.append(1)
        else:
            el4.append(0)

    el1 = np.array(el1)
    el2 = np.array(el2)
    el3 = np.array(el3)
    el4 = np.array(el4)

    el0 = el1+el2+el3+el4
    el = []
    for i in el0:
        if i > 0:
            el.append(1)
        else:
            el.append(0)
    return el

In [27]:
def open_kitchen(df):
    import numpy as np
    el1,el2,el3,el4 = [],[],[],[]
    for i in df['Einbauküche, offene Küche ']:
        if i == 1.0:
            el1.append(1)
        else:
            el1.append(0)
    for i in df['offene Küche ']:
        if i == 1.0:
            el2.append(1)
        else:
            el2.append(0)
    for i in df['Einbauküche, offene Küche, Pantry, Speisekammer ']:
        if i == 1.0:
            el3.append(1)
        else:
            el3.append(0)
    for i in df['Einbauküche, offene Küche, Speisekammer ']:
        if i == 1.0:
            el4.append(1)
        else:
            el4.append(0)

    el1 = np.array(el1)
    el2 = np.array(el2)
    el3 = np.array(el3)
    el4 = np.array(el4)

    el0 = el1+el2+el3+el4
    el = []
    for i in el0:
        if i > 0:
            el.append(1)
        else:
            el.append(0)
    return el

In [28]:
def fitted_kitchen(df):
    import numpy as np
    el1,el2,el3,el4 = [],[],[],[]
    for i in df['Einbauküche ']:
        if i == 1.0:
            el1.append(1)
        else:
            el1.append(0)
    for i in df['Einbauküche, offene Küche ']:
        if i == 1.0:
            el2.append(1)
        else:
            el2.append(0)
    for i in df['Einbauküche, offene Küche, Pantry, Speisekammer ']:
        if i == 1.0:
            el3.append(1)
        else:
            el3.append(0)
    for i in df['Einbauküche, offene Küche, Speisekammer ']:
        if i == 1.0:
            el4.append(1)
        else:
            el4.append(0)

    el1 = np.array(el1)
    el2 = np.array(el2)
    el3 = np.array(el3)
    el4 = np.array(el4)

    el0 = el1+el2+el3+el4
    el = []
    for i in el0:
        if i > 0:
            el.append(1)
        else:
            el.append(0)
    return el

In [29]:
def parkspace(df):
    import numpy as np
    el1,el2,el3 = [],[],[]
    for i in df['1 Stellplatz ']:
        if i == 1.0:
            el1.append(1)
        else:
            el1.append(0)
    for i in df['1 Stellplatz']:
        if i == 1.0:
            el2.append(1)
        else:
            el2.append(0)
    for i in df['Stellplatz']:
        if i == 1.0:
            el3.append(1)
        else:
            el3.append(0)

    el1 = np.array(el1)
    el2 = np.array(el2)
    el3 = np.array(el3)

    el0 = el1+el2+el3
    el = []
    for i in el0:
        if i > 0:
            el.append(1)
        else:
            el.append(0)
    return el

In [30]:
def more_parkspaces(df):
    import numpy as np
    el1,el2 = [],[]
    for i in df['Stellplätze']:
        if i == 1.0:
            el1.append(1)
        else:
            el1.append(0)
    for i in df['2 Stellplätze ']:
        if i == 1.0:
            el2.append(1)
        else:
            el2.append(0)

    el1 = np.array(el1)
    el2 = np.array(el2)

    el0 = el1+el2
    el = []
    for i in el0:
        if i > 0:
            el.append(1)
        else:
            el.append(0)
    return el

### cleaning (3)

In [31]:
lfitted_kitchen = []
lfitted_kitchen = fitted_kitchen(df_raw)
lopen_kitchen = []
lopen_kitchen = open_kitchen(df_raw)
lpantry = []
lpantry = pantry(df_raw)
lparkspace = []
lparkspace = parkspace(df_raw)
lmore_parkspaces = []
lmore_parkspaces = more_parkspaces(df_raw)

In [32]:
df_new["fitted_kitchen"] = lfitted_kitchen
df_new["open_kitchen"] = lopen_kitchen
df_new["pantry"] = lpantry
df_new["parkspace"] = lparkspace
df_new["more_parkspaces"] = lmore_parkspaces

In [33]:
columns_to_drop = ['Einbauküche ','Einbauküche, offene Küche ','offene Küche ','Pantry ',
                    'Pantry, Speisekammer ','Einbauküche, offene Küche, Pantry, Speisekammer ',
                    'Einbauküche, offene Küche, Speisekammer ','1 Stellplatz ','1 Stellplatz',
                    'Stellplatz','Stellplätze', '2 Stellplätze ']
print(df_new.shape[1])
try:
    df_new = df_new.drop(columns = columns_to_drop)
except:
    pass
print(df_new.shape[1])

54
42


In [36]:
c = 0
for i in df_raw.columns:
    if "Speisekammer" in i:
        c+=1
c

3

In [481]:
df_new.columns

Index(['link', 'adresse', 'rent', 'rooms', 'size', 'area', 'rent_range',
       'index', 'not available', 'Ausstattung', 'Zustand',
       'Haustiere erlaubt ', 'Personenaufzug ', 'Baujahr', 'Weitere Räume',
       'Böden', 'Fenster', 'Anschlüsse', 'teilweise klimatisiert ', 'Ausblick',
       'Loggia ', 'Lastenaufzug ', 'Sicherheitstechnik', 'voll klimatisiert ',
       'Wellness', 'Serviceleistungen', 'guest_wc', 'shower', 'bathtub',
       'bathroom_window', 'barrier_free', 'wheelchair_friendly',
       'apartment_sharing', 'balcony', 'garden', 'garden_sharing', 'terrace',
       'fitted_kitchen', 'open_kitchen', 'pantry', 'parkspace',
       'more_parkspaces'],
      dtype='object')

In [37]:
df_new = df_new.rename(columns={'Haustiere erlaubt ': 'pets_allowed', 'Personenaufzug ': 'person_elevator',
                        'Baujahr':'year_constructed','Weitere Räume':'other_rooms','Fenster':'window',
                        'Lastenaufzug ':'service_elevator','Serviceleistungen':'services',
                        'Sicherheitstechnik':'security','Zustand':'condition','Ausstattung':'furnishing',
                        'teilweise klimatisiert ':'partly_air_conditioned','Loggia ':'loggia',
                        'voll klimatisiert ':'fully_air_conditioned','Böden':'floors','Anschlüsse':'connection',
                        'Ausblick':'view','adresse':'addresse','Wellness':'wellness','not available':'not_available'})

In [483]:
df_new.columns

Index(['link', 'addresse', 'rent', 'rooms', 'size', 'area', 'rent_range',
       'index', 'not_available', 'furnishing', 'condition', 'pets_allowed',
       'person_elevator', 'year_constructed', 'other_rooms', 'floors',
       'window', 'connection', 'partly_air_conditioned', 'view', 'loggia',
       'service_elevator', 'security', 'fully_air_conditioned', 'wellness',
       'services', 'guest_wc', 'shower', 'bathtub', 'bathroom_window',
       'barrier_free', 'wheelchair_friendly', 'apartment_sharing', 'balcony',
       'garden', 'garden_sharing', 'terrace', 'fitted_kitchen', 'open_kitchen',
       'pantry', 'parkspace', 'more_parkspaces'],
      dtype='object')

In [38]:
pd.set_option('display.max_columns', None)
df_new.head()

,link,addresse,rent,rooms,size,area,rent_range,index,not_available,furnishing,condition,pets_allowed,person_elevator,year_constructed,other_rooms,floors,window,connection,partly_air_conditioned,view,loggia,service_elevator,security,fully_air_conditioned,wellness,services,guest_wc,shower,bathtub,bathroom_window,barrier_free,wheelchair_friendly,apartment_sharing,balcony,garden,garden_sharing,terrace,fitted_kitchen,open_kitchen,pantry,parkspace,more_parkspaces
0,https://www.immowelt.de/expose/29btl59,Letteallee 64,481.35,2.0,61.24,Reinickendorf,below 600,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,https://www.immowelt.de/expose/29rwk59,Oberschöneweide,724.00,1.0,16.90,Oberschöneweide,600 to 1000,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,https://www.immowelt.de/expose/27xsw57,Mühlenstraße 20,1900.00,2.0,55.00,Friedrichshain,1500 to 2000,0.0,NaN,möbliert,renoviert / saniert,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0
3,https://www.immowelt.de/expose/28u6q5p,Großbeerenstraße 13,3290.00,3.0,115.00,Kreuzberg,over 3000,0.0,NaN,möbliert,renoviert / saniert,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,https://www.immowelt.de/expose/27jyd5w,Stralauer Allee 5,2308.00,2.0,63.00,Friedrichshain,2000 to 3000,0.0,NaN,möbliert,renoviert / saniert,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [41]:
df_new["floors"].unique()

array([nan, 'Parkett, Teppichboden', 'Parkett',
       'Fliesenboden, Teppichboden', 'Fliesenboden, Parkett', 'Laminat',
       'Estrich', 'Fliesenboden', 'Holzdielen',
       'Fliesenboden, Kunststoffboden', 'Fliesenboden, Linoleum',
       'Holzdielen, Fliesenboden', 'Kunststoffboden',
       'Fliesenboden, Laminat', 'Holzdielen, Fliesenboden, Parkett',
       'Fliesenboden, Steinboden', 'Parkett, Steinboden',
       'Fliesenboden, Parkett, Steinboden', 'Marmorboden, Parkett'],
      dtype=object)

In [45]:
def encode_floors(df):
    import numpy as np
    parquet,carpet,tiles,laminate,screed,wooden_floor,synthetic,linoleum,marble = [],[],[],[],[],[],[],[],[]
    for i in df['floors']:
        if "Parkett" in i:
            parquet.append(1)
        else:
            parquet.append(0)
        if "Teppichboden" in i:
            carpet.append(1)
        else:
            carpet.append(0)
        if "Fliesenboden" in i:
            tiles.append(1)
        else:
            tiles.append(0)
        if "Laminat" in i:
            laminate.append(1)
        else:
            laminate.append(0)
        if "Estrich" in i:
            screed.append(1)
        else:
            screed.append(0)
        if "Holzdielen" in i:
            wooden_floor.append(1)
        else:
            wooden_floor.append(0)
        if "Kunststoffboden" in i:
            synthetic.append(1)
        else:
            synthetic.append(0)
        if "Linoleum" in i:
            linoleum.append(1)
        else:
            linoleum.append(0)
        if "Marmorboden" in i:
            marble.append(1)
        else:
            marble.append(0)    
    # parquet,carpet,tiles,laminate,screed,wooden_floor,synthetic,linoleum,marble
    df["parquet"] = parquet
    df["carpet"] = carpet
    df["tiles"] = tiles
    df["laminate"] = laminate
    df["screed"] = screed
    df["wooden_floor"] = wooden_floor
    df["synthetic"] = synthetic
    df["linoleum"] = linoleum
    df["marble"] = marble
    df = df.drop(columns = ["floors"])
    return df

In [47]:
df_new = encode_floors(df_new)

TypeError: argument of type 'float' is not iterable

In [485]:
cols = ["not_available","pets_allowed","person_elevator","other_rooms","floors","window","connection",
        "partly_air_conditioned","view","loggia","service_elevator","security","fully_air_conditioned",
        "wellness","services"]
for i in cols:
    df_new[i].fillna(0, inplace = True)

In [486]:
cols = ["furnishing","condition"]
for i in cols:
    df_new[i].fillna("no information", inplace = True)

In [487]:
furnishing = []
for i in df_new["furnishing"]:
    i = i.replace("möbliert","furnished").replace("neuwertig","like new").replace("teilweise","partly")
    furnishing.append(i)
df_new["furnishing"] = furnishing

In [488]:
condition = []
for i in df_new["condition"]:
    i = i.replace('renoviert / saniert',"renovated").replace("Neubau","new building").replace("Erstbezug","first-time-use").replace("Altbau (bis 1945)","old building").replace("gepflegt","well-maintained")
    condition.append(i)
df_new["condition"] = condition

In [489]:
print(df_new.shape[0])
df_new = df_new[df_new["not_available"] != 1.0]
print(df_new.shape[0])

500
407


In [490]:
cols = ["index","not_available"]
df_new = df_new.drop(columns = cols)

In [491]:
df_new.head()

,link,addresse,rent,rooms,size,area,rent_range,furnishing,condition,pets_allowed,person_elevator,year_constructed,other_rooms,floors,window,connection,partly_air_conditioned,view,loggia,service_elevator,security,fully_air_conditioned,wellness,services,guest_wc,shower,bathtub,bathroom_window,barrier_free,wheelchair_friendly,apartment_sharing,balcony,garden,garden_sharing,terrace,fitted_kitchen,open_kitchen,pantry,parkspace,more_parkspaces
2,https://www.immowelt.de/expose/27xsw57,Mühlenstraße 20,1900.0,2.0,55.0,Friedrichshain,1500 to 2000,furnished,renovated,1.0,1.0,NaN,0,0,0,0,0.0,0,0.0,0.0,0,0.0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0
3,https://www.immowelt.de/expose/28u6q5p,Großbeerenstraße 13,3290.0,3.0,115.0,Kreuzberg,over 3000,furnished,renovated,0.0,0.0,NaN,0,0,0,0,0.0,0,0.0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,https://www.immowelt.de/expose/27jyd5w,Stralauer Allee 5,2308.0,2.0,63.0,Friedrichshain,2000 to 3000,furnished,renovated,1.0,1.0,NaN,0,0,0,0,0.0,0,0.0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
5,https://www.immowelt.de/expose/25m7r5g,Geisbergstraße 31,1500.0,2.0,38.0,Schöneberg,1500 to 2000,furnished,new building,1.0,1.0,2020.0,0,0,0,0,0.0,0,0.0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
6,https://www.immowelt.de/expose/27wkb5r,Schlierseestraße 2,3000.0,3.0,122.8,Grünau,over 3000,like new,no information,0.0,0.0,NaN,0,0,0,0,0.0,0,0.0,0.0,0,0.0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0


### cleaning berlin area

In [492]:
df_new["area"].unique()

array(['Friedrichshain', 'Kreuzberg', 'Schöneberg', 'Grünau', 'Kaulsdorf',
       'Westend', 'Prenzlauer Berg', 'Friedrichshagen', 'Oberschöneweide',
       'Tiergarten', 'Lichtenrade', 'Wannsee', 'Mariendorf',
       'Wilmersdorf', 'Gesundbrunnen', 'Charlottenburg', 'Dahlem',
       'Karolinenhof', 'Neukölln', 'Moabit', 'Mitte', 'Johannisthal',
       'Alt-Hohenschönhausen', 'Weißensee', 'Reinickendorf', 'Friedenau',
       'Köpenick', 'Französisch Buchholz', 'Spandau', 'Adlershof',
       'Baumschulenweg', 'Steglitz', 'Lichterfelde', 'Pankow',
       'Blankenburg', 'Lankwitz', 'Hellersdorf', 'Lichtenberg',
       'Biesdorf', 'Rahnsdorf', 'Karow', 'Friedrichsfelde', 'Wedding',
       'Kladow', 'Schmargendorf', 'Mahlsdorf', 'Heinersdorf',
       'Niederschönhausen', 'Alt-Treptow', 'Wittenau', 'Britz',
       'Karlshorst', 'Marienfelde', 'Hermsdorf', 'Tempelhof', 'Halensee',
       'Tegel', 'Neu-Hohenschönhausen', 'Zehlendorf', 'Schmöckwitz',
       'Waidmannslust', 'Heiligensee', 'Märk

In [493]:
def berlin_area(df):
    mitte = ["Mitte","Moabit","Hansaviertel","Tiergarten",
            "Wedding","Gesundbrunnen"]
    fr_kberg = ["Friedrichshain","Kreuzberg"]
    pankow = ["Prenzlauer Berg","Weißensee","Blankenburg",
            "Heinersdorf","Karow","Stadtrandsiedlung Malchow",
            "Pankow","Blankenfelde","Buch","Französisch Buchholz",
            "Niederschönhausen","Rosenthal","Wilhelmsruh"]
    charl_w = ["Charlottenburg","Wilmersdorf","Schmargendorf",
            "Grunewald","Westend","Charlottenburg-Nord","Halensee"]
    spandau = ["Spandau","Haselhorst","Siemensstadt","Staaken",
            "Gatow","Kladow","Hakenfelde","Falkenhagener Feld",
            "Wilhelmstadt"]
    stegl_z = ["Steglitz","Lichterfelde","Lankwitz","Zehlendorf",
            "Dahlem","Nikolassee","Wannsee","Schlachtensee"]
    temp_sch = ["Schöneberg","Friedenau","Tempelhof","Mariendorf",
            "Marienfelde","Lichtenrade"]
    neukln = ["Neukölln","Britz","Buckow","Rudow","Gropiusstadt"]
    trept_k = ["Alt-Treptow","Plänterwald","Baumschulenweg",
            "Johannisthal","Niederschöneweide","Altglienicke",
            "Adlershof","Bohnsdorf","Oberschöneweide","Köpenick",
            "Friedrichshagen","Rahnsdorf","Grünau","Müggelheim",
            "Schmöckwitz","Karolinenhof"]
    mar_hel = ["Marzahn","Biesdorf","Kaulsdorf","Mahlsdorf",
            "Hellersdorf"]
    licht = ["Friedrichsfelde","Karlshorst","Lichtenberg","Falkenberg",
            "Malchow","Wartenberg","Neu-Hohenschönhausen",
            "Alt-Hohenschönhausen","Fennpfuhl","Rummelsburg"]
    rein = ["Reinickendorf","Tegel","Konradshöhe","Heiligensee",
            "Frohnau","Hermsdorf","Waidmannslust","Lübars",
            "Wittenau","Märkisches Viertel","Borsigwalde"]
    area = []
    for i in df["area"]:
        if i in mitte:
            area.append("Mitte")
        elif i in fr_kberg:
            area.append("Friedrichshain-Kreuzberg")
        elif i in pankow:
            area.append("Pankow")
        elif i in charl_w:
            area.append("Charlottenburg-Wilmersdorf")
        elif i in spandau:
            area.append("Spandau")
        elif i in stegl_z:
            area.append("Steglitz-Zehlendorf")
        elif i in temp_sch:
            area.append("Tempelhof-Schöneberg")
        elif i in neukln:
            area.append("Neukölln")
        elif i in trept_k:
            area.append("Treptow-Köpenick")
        elif i in mar_hel:
            area.append("Marzahn-Hellersdorf")
        elif i in licht:
            area.append("Lichtenberg")
        elif i in rein:
            area.append("Reinickendorf")
        else:
            area.append(i)
    return area

In [494]:
area = berlin_area(df_new)
df_new["area"] = area

In [495]:
df_new.area.unique()

array(['Friedrichshain-Kreuzberg', 'Tempelhof-Schöneberg',
       'Treptow-Köpenick', 'Marzahn-Hellersdorf',
       'Charlottenburg-Wilmersdorf', 'Pankow', 'Mitte',
       'Steglitz-Zehlendorf', 'Neukölln', 'Lichtenberg', 'Reinickendorf',
       'Spandau'], dtype=object)

### cleaning year_constructed

In [496]:
def year(row):
    if (row["condition"]=="old building") and (row["year_constructed"]=="NaN"):
        return "1900"
    elif (row["condition"]=="new building") and (row["year_constructed"]=="NaN"):
        return "2020"
    elif (row["year_constructed"]==1023):
        return "2023"
    else:
        return row["year_constructed"]

In [497]:
df_new["year_constructed"] = df_new["year_constructed"].fillna("NaN")

In [498]:
import numpy as np
df_new["year"] = df_new.apply(year, axis = 1)


In [499]:
df_new["year"][df_new["year"]!="NaN"].astype("int64").mean()

1981.085294117647

In [500]:
year = []
for i in df_new["year"]:
    if i == "NaN":
        year.append("1975")
    else:
        year.append(i)
df_new["year"] = year


In [501]:
df_new["year"].value_counts()

1975      67
2022.0    55
1900.0    36
2023.0    31
2020.0    21
          ..
1888.0     1
1912.0     1
1927.0     1
1965.0     1
1967.0     1
Name: year, Length: 73, dtype: int64

In [502]:
df_new = df_new.drop(columns = ["year_constructed"])
df_new["year"] = df_new["year"].astype("int64")
df_new.head()

,link,addresse,rent,rooms,size,area,rent_range,furnishing,condition,pets_allowed,person_elevator,other_rooms,floors,window,connection,partly_air_conditioned,view,loggia,service_elevator,security,fully_air_conditioned,wellness,services,guest_wc,shower,bathtub,bathroom_window,barrier_free,wheelchair_friendly,apartment_sharing,balcony,garden,garden_sharing,terrace,fitted_kitchen,open_kitchen,pantry,parkspace,more_parkspaces,year
2,https://www.immowelt.de/expose/27xsw57,Mühlenstraße 20,1900.0,2.0,55.0,Friedrichshain-Kreuzberg,1500 to 2000,furnished,renovated,1.0,1.0,0,0,0,0,0.0,0,0.0,0.0,0,0.0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1975
3,https://www.immowelt.de/expose/28u6q5p,Großbeerenstraße 13,3290.0,3.0,115.0,Friedrichshain-Kreuzberg,over 3000,furnished,renovated,0.0,0.0,0,0,0,0,0.0,0,0.0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1975
4,https://www.immowelt.de/expose/27jyd5w,Stralauer Allee 5,2308.0,2.0,63.0,Friedrichshain-Kreuzberg,2000 to 3000,furnished,renovated,1.0,1.0,0,0,0,0,0.0,0,0.0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1975
5,https://www.immowelt.de/expose/25m7r5g,Geisbergstraße 31,1500.0,2.0,38.0,Tempelhof-Schöneberg,1500 to 2000,furnished,new building,1.0,1.0,0,0,0,0,0.0,0,0.0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,2020
6,https://www.immowelt.de/expose/27wkb5r,Schlierseestraße 2,3000.0,3.0,122.8,Treptow-Köpenick,over 3000,like new,no information,0.0,0.0,0,0,0,0,0.0,0,0.0,0.0,0,0.0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1975


In [503]:
cols = ["partly_air_conditioned","loggia","service_elevator","fully_air_conditioned"]
for i in cols:
    df_new[i] = df_new[i].astype("int64")

In [504]:
rent = []
for i in df_new["rent"]:
    if i < 100:
        i = i*10
        rent.append(i)
    else:
        rent.append(i)
df_new["rent"] = rent

In [505]:
df_new["rent"].min()

248.71

In [506]:
df_new.isna().sum()

link                      0
addresse                  0
rent                      0
rooms                     0
size                      0
area                      0
rent_range                0
furnishing                0
condition                 0
pets_allowed              0
person_elevator           0
other_rooms               0
floors                    0
window                    0
connection                0
partly_air_conditioned    0
view                      0
loggia                    0
service_elevator          0
security                  0
fully_air_conditioned     0
wellness                  0
services                  0
guest_wc                  0
shower                    0
bathtub                   0
bathroom_window           0
barrier_free              0
wheelchair_friendly       0
apartment_sharing         0
balcony                   0
garden                    0
garden_sharing            0
terrace                   0
fitted_kitchen            0
open_kitchen        

In [507]:
df_new["year"].min()

1848

In [508]:
df_new.head()

,link,addresse,rent,rooms,size,area,rent_range,furnishing,condition,pets_allowed,person_elevator,other_rooms,floors,window,connection,partly_air_conditioned,view,loggia,service_elevator,security,fully_air_conditioned,wellness,services,guest_wc,shower,bathtub,bathroom_window,barrier_free,wheelchair_friendly,apartment_sharing,balcony,garden,garden_sharing,terrace,fitted_kitchen,open_kitchen,pantry,parkspace,more_parkspaces,year
2,https://www.immowelt.de/expose/27xsw57,Mühlenstraße 20,1900.0,2.0,55.0,Friedrichshain-Kreuzberg,1500 to 2000,furnished,renovated,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1975
3,https://www.immowelt.de/expose/28u6q5p,Großbeerenstraße 13,3290.0,3.0,115.0,Friedrichshain-Kreuzberg,over 3000,furnished,renovated,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1975
4,https://www.immowelt.de/expose/27jyd5w,Stralauer Allee 5,2308.0,2.0,63.0,Friedrichshain-Kreuzberg,2000 to 3000,furnished,renovated,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1975
5,https://www.immowelt.de/expose/25m7r5g,Geisbergstraße 31,1500.0,2.0,38.0,Tempelhof-Schöneberg,1500 to 2000,furnished,new building,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,2020
6,https://www.immowelt.de/expose/27wkb5r,Schlierseestraße 2,3000.0,3.0,122.8,Treptow-Köpenick,over 3000,like new,no information,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1975


In [509]:
df_new.to_csv("data full clean.csv", index = False)